In [1]:
#step1: Create & Setup hugging face API token in Colab

In [1]:
# Install unsloth with CUDA support
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# Install compatible versions based on unsloth-zoo requirements
!pip install "transformers>=4.51.3,<4.57.2" "datasets>=3.4.1,<4.4.0" "trl>=0.18.2,<=0.24.0" --upgrade

# Install other required packages
!pip install accelerate bitsandbytes wandb --upgrade

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-slhadoat/unsloth_360ebc65ab77432c84d3585c2796bf3a
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-slhadoat/unsloth_360ebc65ab77432c84d3585c2796bf3a
  Resolved https://github.com/unslothai/unsloth.git to commit bda9e3d39b425f902d29e80c1f2870be7048d9c3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.4/284.4 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 83.7 MB/s eta 0:00:00


In [2]:

# Step3: Import necessary libraries
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from unsloth import is_bfloat16_supported
from huggingface_hub import login
from transformers import TrainingArguments
from datasets import load_dataset
import wandb

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:

# Step4: Check HF token
from google.colab import userdata
hf_token = userdata.get('HF_ACCESS_TOKEN')
login(hf_token)

In [4]:

# Optional: Check GPU availability
# Test if CUDA is available
import torch
print("CUDA available:", torch.cuda.is_available())
print("GPU device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

CUDA available: True
GPU device: Tesla T4


In [8]:

# Step5: Setup pretrained DeepSeek-R1

model_name = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
max_sequence_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_sequence_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = hf_token
)

==((====))==  Unsloth 2025.11.4: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [9]:
# Step6: Setup system prompt
prompt_style = """
Below is a task description along with additional context provided in the input section. Your goal is to provide a well-reasoned response that effectively addresses the request.

Before crafting your answer, take a moment to carefully analyze the question. Develop a clear, step-by-step thought process to ensure your response is both logical and accurate.

### Task:
You are a medical expert specializing in clinical reasoning, diagnostics, and treatment planning. Answer the medical question below using your advanced knowledge.

### Query:
{}

### Answer:
{}
"""

In [10]:

# Step7: Run Inference on the model

# Define a test question
question = """A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or
              sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings,
              what would cystometry most likely reveal about her residual volume and detrusor contractions?"""

FastLanguageModel.for_inference(model)

# Tokenize the input
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

# Generate a response
outputs = model.generate (
    input_ids = inputs.input_ids,
    attention_mask = inputs.attention_mask,
    max_new_tokens = 1200,
    use_cache = True
)

# Decode the response tokens back to text
response = tokenizer.batch_decode(outputs)


print(response)



["<｜begin▁of▁sentence｜>\nBelow is a task description along with additional context provided in the input section. Your goal is to provide a well-reasoned response that effectively addresses the request.\n\nBefore crafting your answer, take a moment to carefully analyze the question. Develop a clear, step-by-step thought process to ensure your response is both logical and accurate.\n\n### Task:\nYou are a medical expert specializing in clinical reasoning, diagnostics, and treatment planning. Answer the medical question below using your advanced knowledge.\n\n### Query:\nA 61-year-old woman with a long history of involuntary urine loss during activities like coughing or\n              sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings,\n              what would cystometry most likely reveal about her residual volume and detrusor contractions?\n\n### Answer:\n\n**Step-by-Step Explanation:**\n\n1. **Understanding the Symptoms:**\n   The 

In [11]:

print(response[0].split("### Answer:")[1])



**Step-by-Step Explanation:**

1. **Understanding the Symptoms:**
   The patient is a 61-year-old woman with a history of involuntary urine loss during activities such as coughing or sneezing. She doesn't experience leakage at night, which is a key point because it suggests that her bladder doesn't leak when it's full, possibly indicating that her detrusor muscle isn't contracting inappropriately during sleep.

2. **Gynecological Exam and Q-Tip Test Findings:**
   - **Gynecological Exam:** Likely focused on the bladder and urethra. If there's a problem with the urethra, it could cause urinary incontinence. If the exam doesn't show abnormalities, it suggests that the issue isn't related to the urethra but is more likely a bladder problem.
   - **Q-Tip Test:** This test involves inserting a Q-tip catheter into the urethra to measure the pressure and muscle contractions in the detrusor muscle. If the detrusor muscle is overactive (hypersensitive), it might contract too strongly, leading

In [12]:

# Step8: Setup fine-tuning

# Load Dataset
medical_dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", "en", split = "train[:500]", trust_remote_code = True)

`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'FreedomIntelligence/medical-o1-reasoning-SFT' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.
ERROR:datasets.load:`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'FreedomIntelligence/medical-o1-reasoning-SFT' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.


Generating train split:   0%|          | 0/19704 [00:00<?, ? examples/s]

In [13]:

medical_dataset[1]

{'Question': 'A 33-year-old woman is brought to the emergency department 15 minutes after being stabbed in the chest with a screwdriver. Given her vital signs of pulse 110/min, respirations 22/min, and blood pressure 90/65 mm Hg, along with the presence of a 5-cm deep stab wound at the upper border of the 8th rib in the left midaxillary line, which anatomical structure in her chest is most likely to be injured?',
 'Complex_CoT': "Okay, let's figure out what's going on here. A woman comes in with a stab wound from a screwdriver. It's in her chest, upper border of the 8th rib, left side, kind of around the midaxillary line. First thought, that's pretty close to where the lung sits, right?\n\nLet's talk about location first. This spot is along the left side of her body. Above the 8th rib, like that, is where a lot of important stuff lives, like the bottom part of the left lung, possibly the diaphragm too, especially considering how deep the screwdriver went.\n\nThe wound is 5 cm deep. Tha

In [14]:

EOS_TOKEN = tokenizer.eos_token  # Define EOS_TOKEN which tells the model when to stop generating text during training
EOS_TOKEN

'<｜end▁of▁sentence｜>'

In [15]:

### Finetuning
# Updated training prompt style to add  tag
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.
Please answer the following medical question.

### Question:
{}

### Response:

{}

{}"""



In [16]:

# Prepare the data for fine-tuning

def preprocess_input_data(examples):
  inputs = examples["Question"]
  cots = examples["Complex_CoT"]
  outputs = examples["Response"]

  texts = []

  for input, cot, output in zip(inputs, cots, outputs):
    text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
    texts.append(text)

  return {
      "texts" : texts,
  }

In [17]:

finetune_dataset = medical_dataset.map(preprocess_input_data, batched = True)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [19]:

finetune_dataset["texts"][1]

"Below is an instruction that describes a task, paired with an input that provides further context.\nWrite a response that appropriately completes the request.\nBefore answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.\n\n### Instruction:\nYou are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.\nPlease answer the following medical question.\n\n### Question:\nA 33-year-old woman is brought to the emergency department 15 minutes after being stabbed in the chest with a screwdriver. Given her vital signs of pulse 110/min, respirations 22/min, and blood pressure 90/65 mm Hg, along with the presence of a 5-cm deep stab wound at the upper border of the 8th rib in the left midaxillary line, which anatomical structure in her chest is most likely to be injured?\n\n### Response:\n\nOkay, let's figure out what's going on here. A woman comes in with a stab wound fro

In [20]:

# Step9: Setup/Apply LoRA finetuning to the model

model_lora = FastLanguageModel.get_peft_model(
    model = model,
    r = 16,
    target_modules = [
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj"
    ],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3047,
    use_rslora = False,
    loftq_config = None
)

Unsloth 2025.11.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [21]:

# Add this before creating the trainer
if hasattr(model, '_unwrapped_old_generate'):
    del model._unwrapped_old_generate

In [30]:
# SOLUTION 1: Explicit list handling with validation
def formatting_func(examples):
    """
    Ensures we always return a proper list of strings.
    """
    texts = examples['texts']

    # Validate and ensure it's a list
    if not isinstance(texts, list):
        texts = [texts]

    # Ensure all items are strings
    texts = [str(text) for text in texts]

    return texts

# SOLUTION 2: If above doesn't work, try this alternative approach
# Remove dataset_text_field and use packing=False
def formatting_func_alt(examples):
    """
    Alternative formatting with explicit iteration.
    """
    output = []

    # Handle both single examples and batches
    if isinstance(examples['texts'], list):
        # This is a batch
        for text in examples['texts']:
            output.append(str(text))
    else:
        # Single example
        output.append(str(examples['texts']))

    return output

# Now create the trainer with the formatting_func
trainer = SFTTrainer(
    model = model_lora,
    tokenizer = tokenizer,
    train_dataset = finetune_dataset,

    # Add the formatting function
    formatting_func = formatting_func,

    max_seq_length = max_sequence_length,
    dataset_num_proc = 1,

    # CRITICAL: Add packing parameter
    packing = False,

    # Define training args
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        num_train_epochs = 1,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

# Optional: Test your formatting function BEFORE creating trainer
print("=" * 60)
print("DEBUGGING FORMATTING FUNCTION")
print("=" * 60)

# Test with different batch sizes
for batch_size in [1, 2, 4]:
    print(f"\nTesting with batch_size={batch_size}")
    sample = finetune_dataset[:batch_size]
    result = formatting_func(sample)

    print(f"  Input type: {type(sample)}")
    print(f"  Input 'texts' type: {type(sample['texts'])}")
    print(f"  Output type: {type(result)}")
    print(f"  Output length: {len(result)}")
    print(f"  First item type: {type(result[0])}")
    print(f"  First item length: {len(result[0])} chars")

    # Validate output
    assert isinstance(result, list), f"Output must be list, got {type(result)}"
    assert all(isinstance(item, str) for item in result), "All items must be strings"
    print("  ✅ Validation passed!")

print("\n" + "=" * 60)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/500 [00:00<?, ? examples/s]

DEBUGGING FORMATTING FUNCTION

Testing with batch_size=1
  Input type: <class 'dict'>
  Input 'texts' type: <class 'list'>
  Output type: <class 'list'>
  Output length: 1
  First item type: <class 'str'>
  First item length: 3035 chars
  ✅ Validation passed!

Testing with batch_size=2
  Input type: <class 'dict'>
  Input 'texts' type: <class 'list'>
  Output type: <class 'list'>
  Output length: 2
  First item type: <class 'str'>
  First item length: 3035 chars
  ✅ Validation passed!

Testing with batch_size=4
  Input type: <class 'dict'>
  Input 'texts' type: <class 'list'>
  Output type: <class 'list'>
  Output length: 4
  First item type: <class 'str'>
  First item length: 3035 chars
  ✅ Validation passed!



In [31]:
# Setup WANDB
from google.colab import userdata
wnb_token = userdata.get("WANDB_API_TOKEN")
# Login to WnB
wandb.login(key=wnb_token) # import wandb
run = wandb.init(
    project='Fine-tune-DeepSeek-R1-on-Medical-CoT-Dataset',
    job_type="training",
    anonymous="allow"
)


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sahilkatiyar2024 (sahilkatiyar2024-mumbai-university-and-idol-news) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


In [32]:

# Start the fine-tuning process
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 500 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,1.944400
20,1.445000
30,1.421300
40,1.376500
50,1.401500
60,1.380400


train/epoch,▁▂▄▅▇██
train/global_step,▁▂▄▅▇██
train/grad_norm,█▂▁▁▂▁
train/learning_rate,█▇▅▄▂▁
train/loss,█▂▂▁▁▁
total_flos,1.6602988216516608e+16
train/epoch,0.96
train/global_step,60
train/grad_norm,0.25833
train/learning_rate,0.0
train/loss,1.3804


In [33]:

wandb.finish()


In [34]:
# Step10: Testing after fine-tuning
question = """A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing
              but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings,
              what would cystometry most likely reveal about her residual volume and detrusor contractions?"""

FastLanguageModel.for_inference(model_lora)

# Tokenize the input
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

# Generate a response
outputs = model_lora.generate (
    input_ids = inputs.input_ids,
    attention_mask = inputs.attention_mask,
    max_new_tokens = 1200,
    use_cache = True
)

# Decode the response tokens back to text
response = tokenizer.batch_decode(outputs)

print(response)

["<｜begin▁of▁sentence｜>\nBelow is a task description along with additional context provided in the input section. Your goal is to provide a well-reasoned response that effectively addresses the request.\n\nBefore crafting your answer, take a moment to carefully analyze the question. Develop a clear, step-by-step thought process to ensure your response is both logical and accurate.\n\n### Task:\nYou are a medical expert specializing in clinical reasoning, diagnostics, and treatment planning. Answer the medical question below using your advanced knowledge.\n\n### Query:\nA 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing\n              but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings,\n              what would cystometry most likely reveal about her residual volume and detrusor contractions?\n\n### Answer:\n\nAlright, let's think about this. This woman is 61 and has been dealing with 

In [35]:

print(response[0].split("### Answer:")[1])



Alright, let's think about this. This woman is 61 and has been dealing with involuntary urine loss for a long time, especially when she coughs or sneezes. That's a classic sign of urinary incontinence, probably due to an overactive bladder. 

Now, she's done a Q-tip test, which is often used to check for urethral obstruction. If the Q-tip test is positive, it suggests that there's something blocking the urethra, which might lead to difficulty in passing urine, but in this case, she's been having involuntary losses, not obstructive symptoms. So, it seems like the Q-tip test didn't pick up anything, meaning the urethra isn't really the issue here. 

That makes me think about the bladder itself. Given her history, it's likely that the bladder is overactive, causing her to lose urine involuntarily. Now, when we think about cystometry, it's like a stress test for the bladder. It shows how the bladder behaves when it's full. 

In cystometry, if there's an overactive bladder, we might see s

In [37]:

question = """A 59-year-old man presents with a fever, chills, night sweats, and generalized fatigue,
              and is found to have a 12 mm vegetation on the aortic valve. Blood cultures indicate gram-positive, catalase-negative,
              gamma-hemolytic cocci in chains that do not grow in a 6.5% NaCl medium.
              What is the most likely predisposing factor for this patient's condition?"""

FastLanguageModel.for_inference(model_lora)

# Tokenize the input
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

# Generate a response
outputs = model_lora.generate (
    input_ids = inputs.input_ids,
    attention_mask = inputs.attention_mask,
    max_new_tokens = 1200,
    use_cache = True
)

# Decode the response tokens back to text
response = tokenizer.batch_decode(outputs)

print(response[0].split("### Answer:")[1])

question = """A 59-year-old man presents with a fever, chills, night sweats, and generalized fatigue,
              and is found to have a 12 mm vegetation on the aortic valve. Blood cultures indicate gram-positive, catalase-negative,
              gamma-hemolytic cocci in chains that do not grow in a 6.5% NaCl medium.
              What is the most likely predisposing factor for this patient's condition?"""

FastLanguageModel.for_inference(model_lora)

# Tokenize the input
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

# Generate a response
outputs = model_lora.generate (
    input_ids = inputs.input_ids,
    attention_mask = inputs.attention_mask,
    max_new_tokens = 1200,
    use_cache = True
)

# Decode the response tokens back to text
response = tokenizer.batch_decode(outputs)

print(response[0].split("### Answer:")[1])




Alright, let's dive into this case. We're dealing with a 59-year-old man who's got some pretty classic symptoms of a bacterial infection, like fever, chills, and night sweats. That's definitely a red flag for something like endocarditis, which I know can be pretty serious if it's not caught early.

Now, let's look at what the lab work says. The blood cultures show gram-positive, catalase-negative, gamma-hemolytic cocci in chains. Hmm, that's interesting. These look like they could be a type of Streptococcus, specifically maybe Streptococcus mitis or some variant. But here's the kicker: they don't grow in a 6.5% NaCl medium. That's a clue because Streptococcus mitis usually doesn't grow under those conditions. Oh, and wait, I think Streptococcus mitis is known for being particularly virulent when it's in the heart and can cause endocarditis, especially in older folks.

So, we're piecing this together. The symptoms are pointing towards endocarditis, and the lab results are matching up 

In [36]:
# Step 10: Testing after fine-tuning

# Define your test question
question = """A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing
but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings,
what would cystometry most likely reveal about her residual volume and detrusor contractions?"""

# Define the prompt template (match your training format)
prompt_template = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.
Please answer the following medical question.

### Question:
{}

### Response:
"""

# Format the prompt
formatted_prompt = prompt_template.format(question)

# Set model to inference mode
FastLanguageModel.for_inference(model_lora)

# Tokenize the input
inputs = tokenizer(
    [formatted_prompt],
    return_tensors="pt",
    truncation=True,
    max_length=max_sequence_length
).to("cuda")

print("Generating response...\n")

# Generate a response
outputs = model_lora.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
    temperature=0.7,  # Adjust for creativity (0.1-1.0)
    top_p=0.9,        # Nucleus sampling
    do_sample=True,   # Enable sampling for more natural responses
    pad_token_id=tokenizer.eos_token_id,
)

# Decode the response
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)

# Print the full response
print("="*80)
print("FULL OUTPUT:")
print("="*80)
print(response[0])
print("="*80)

# Extract only the generated response (remove the prompt)
generated_text = response[0]
if "### Response:" in generated_text:
    answer_only = generated_text.split("### Response:")[-1].strip()
    print("\n" + "="*80)
    print("ANSWER ONLY:")
    print("="*80)
    print(answer_only)
    print("="*80)

# Optional: Test multiple questions
print("\n\n" + "="*80)
print("TESTING MULTIPLE QUESTIONS")
print("="*80)

test_questions = [
    "What is the most common cause of bacterial meningitis in adults?",
    "A patient presents with chest pain radiating to the left arm. What is the first test you would order?",
    question  # Your original question
]

for idx, test_q in enumerate(test_questions, 1):
    print(f"\n--- Question {idx} ---")
    print(test_q[:100] + "..." if len(test_q) > 100 else test_q)

    formatted = prompt_template.format(test_q)
    inputs = tokenizer([formatted], return_tensors="pt").to("cuda")

    outputs = model_lora.generate(
        **inputs,
        max_new_tokens=500,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
    )

    result = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract answer
    if "### Response:" in result:
        answer = result.split("### Response:")[-1].strip()
        # Truncate for display
        print(f"Answer: {answer[:200]}..." if len(answer) > 200 else f"Answer: {answer}")
    print()

Generating response...

FULL OUTPUT:
Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.
Please answer the following medical question.

### Question:
A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing
but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings,
what would cystometry most likely reveal about her residual volume and detrusor contractions?

### Response:
Okay, let's think about this. We're dealing with a woman who has been dealing with some bladder issues for quite a while. She's been having involuntary urine loss